In [1]:
import sys
sys.path.append('SceneGraphParser')
import sng_parser 

from tqdm import tqdm

In [2]:
import os
import h5py

label_h5 = os.path.join('../../../msrvtt_captioning/output/metadata', 'msrvtt_train_sequencelabel.h5') 
label_h5 = h5py.File(label_h5, 'r')
vocab = [i for i in label_h5['vocab']]

In [3]:
import json

train_val_file = json.load(open('../files/train_videodatainfo.json'))
test_file = json.load(open('../files/test_videodatainfo.json'))

In [4]:
entity_list = []
rel_list = []
for i, cap in enumerate(tqdm(train_val_file['sentences'])):
    graph = sng_parser.parse(cap['caption'])
    for entity in graph['entities']:
        if entity['lemma_head'].encode() in vocab:
            entity_list.append(entity['lemma_head'])
        elif entity['head'].encode() in vocab:
            entity_list.append(entity['head'])
    for rel in graph['relations']:
        if rel['lemma_relation'].encode() in vocab:
            rel_list.append(rel['lemma_relation'])
        elif rel['relation'].encode() in vocab:
            rel_list.append(rel['relation'])

100%|██████████| 140200/140200 [20:03<00:00, 116.52it/s]


In [5]:
from collections import Counter
from PyDictionary import PyDictionary
dictionary=PyDictionary()

entity_freq = Counter(entity_list)
entity_freq.most_common()
dictionary.synonym('men')

['personnel',
 'manpower',
 'work party',
 'gang',
 'shift',
 'full complement',
 'workforce',
 'hands',
 'work force',
 'force',
 'crew',
 'complement']

In [6]:
entity_list_1 = []
for i, entity in enumerate(tqdm(entity_list)):
#     import pdb; pdb.set_trace()
    if len(entity.split(' ')) > 0:
        entity_list_1.append(entity.split(' ')[-1])
    else:
        entity_list_1.append(entity)

100%|██████████| 312529/312529 [00:00<00:00, 918961.52it/s] 


In [7]:
entity_freq = Counter(entity_list_1)
# entity_freq = entity_freq.most_common()
entity_freq_ = entity_freq.most_common(5220)    # freq >= 3
all_ent = [a[0] for a in entity_freq_]

In [8]:
import spacy
nlp = spacy.load('en_core_web_lg')
# [t.lemma_ for t in nlp('women')][0]

In [9]:
synonym_list = {}
for i, item in enumerate(all_ent):
    cluster = [t.cluster for t in nlp(item)][0]
    if cluster not in synonym_list:
        synonym_list[cluster] = [item]
    else:
        synonym_list[cluster].append(item)

In [11]:
synonyms = []
synonyms_freq = []
keys = list(synonym_list.keys())
synonyms.append(synonym_list[keys[0]])
synonyms_freq.append([entity_freq_[all_ent.index(ent)][1] for ent in synonym_list[keys[0]]])
synonyms.append(synonym_list[keys[1]])
synonyms_freq.append([entity_freq_[all_ent.index(ent)][1] for ent in synonym_list[keys[1]]])

synonym_all = synonyms[0].copy()
synonym_all.extend([a for a in synonyms[-1]])

In [12]:
all_ent_copy = all_ent.copy()

In [15]:
for ent, freq in entity_freq_:
    if len(synonyms) == 60:
        break
    if ent in synonym_all:
        continue
    else:
        try:
            ent_syn = dictionary.synonym(ent) 
            rest_ents = list(set(all_ent_copy)-set(synonym_all))
            intersection = list(set(rest_ents).intersection(set(ent_syn)))
            intersection.append(ent)
            synonyms.append(intersection)
            synonyms_freq.append([entity_freq_[all_ent.index(ent)][1] for ent in intersection])
            synonym_all.extend([a for a in synonyms[-1]])
        except:
            continue            


that has no Synonyms in the API
which has no Synonyms in the API
what has no Synonyms in the API


In [17]:
import random
random.choices(synonyms[50], synonyms_freq[50], k=1)

['floor']

In [18]:
index = 58
print(synonyms[index])
print(synonyms_freq[index])

['heat', 'skeleton', 'courtyard', 'nursery', 'club', 'court', 'lobby', 'hall', 'shaft', 'upstairs', 'stairway', 'theater', 'hotel', 'center', 'corner', 'skyscraper', 'temple', 'structure', 'restaurant', 'theatre', 'staircase', 'architecture', 'library', 'construction', 'wall', 'level', 'clubhouse', 'tavern', 'centre', 'elevator', 'school', 'lift', 'rink', 'building']
[26, 49, 12, 2, 49, 238, 7, 66, 1, 2, 4, 22, 35, 27, 75, 8, 22, 38, 197, 7, 27, 3, 68, 35, 464, 46, 1, 3, 2, 42, 225, 11, 4, 540]


In [19]:
rel_freq = Counter(rel_list)
rel_freq = rel_freq.most_common(9)
rels = [a[0] for a in rel_freq]

In [21]:
synonyms_class = {}
for i, sym_ in enumerate(synonyms):
    for sym in sym_:
        synonyms_class[sym] = i + 1

In [20]:
len(synonyms)

60

In [62]:
vocab.append('<ent>'.encode())
vocab.append('<rel>'.encode())

wordtoix = {w: i for i, w in enumerate(vocab)}
ixtoword = {i: w for i, w in enumerate(vocab)}

captions = label_h5['labels'][:]

In [63]:
ent_rel_list = []
for _, cap in enumerate(tqdm(captions)):
    cap = cap[1:]
    ent_rel = {'ent': [], 'rel': []}
    if 0 in cap:
        eos_pos = list(cap).index(0)
        cap = cap[:eos_pos]
    cap_tokens = [ixtoword[a].decode() for a in cap]
#     if '<unk>' in cap_tokens:
#         cap_tokens.remove('<unk>')
    cap_words = ' '.join(cap_tokens)
    
    graph = sng_parser.parse(cap_words)
    for entity in graph['entities']:
        ent = entity['lemma_head']
        if ent in synonym_all:
            pos = [synonyms.index(a) for a in synonyms if ent in a][0]
            ent_rel['ent'].append(pos)
        else:
            ent_rel['ent'].append(len(synonyms))
            
    for relation in graph['relations']:

        rel = relation['lemma_relation']
        if rel in rels:
            ent_rel['rel'].append(rels.index(rel))
        else:
            ent_rel['rel'].append(len(rels))
            
    ent_rel_list.append(ent_rel)
#     import pdb; pdb.set_trace()


100%|██████████| 130260/130260 [15:40<00:00, 138.43it/s]


In [65]:
ent_rel_token_list = []
for _, cap in enumerate(tqdm(captions)):
    cap = cap[1:]
    ent_rel = {'ent': [], 'rel': []}
    if 0 in cap:
        eos_pos = list(cap).index(0)
        cap = cap[:eos_pos]
    cap_tokens = [ixtoword[a].decode() for a in cap]
    cap_words = ' '.join(cap_tokens)
    
    graph = sng_parser.parse(cap_words)
    for entity in graph['entities']:
        if entity['lemma_head'].encode() in vocab:
            ent_rel['ent'].append(entity['lemma_head'])
        else:
            ent_rel['ent'].append(entity['head'])

            
    for relation in graph['relations']:
        
        if relation['lemma_relation'].encode() in vocab:
            ent_rel['rel'].append(relation['lemma_relation'])
        else:
            ent_rel['rel'].append(relation['relation'])

            
    ent_rel_token_list.append(ent_rel)

100%|██████████| 130260/130260 [16:06<00:00, 134.71it/s]


In [ ]:
pickle.dump(ent_rel_list, open('../files/ent_rel_list.pkl', 'wb'))

In [22]:
# synonyms_freq_list = {'synonyms': synonyms, 'synonyms_freq': synonyms_freq, 'synonyms_class': synonyms_class, 'rels': rels}
synonyms_freq_list = {'synonyms': synonyms, 'synonyms_freq': synonyms_freq, 'synonyms_class': synonyms_class}

import pickle
pickle.dump(synonyms_freq_list, open('../files/synonyms_freq_list_60.pkl', 'wb'))

In [66]:
ent_rel_token_list

[{'ent': ['car'], 'rel': []},
 {'ent': ['group'], 'rel': []},
 {'ent': ['man', 'vehicle', 'countryside'], 'rel': ['drive', 'through']},
 {'ent': ['man', 'road', 'audi'], 'rel': ['drive', 'in']},
 {'ent': ['man', 'car'], 'rel': ['drive']},
 {'ent': ['man', 'car'], 'rel': ['drive']},
 {'ent': ['man', 'road'], 'rel': ['down']},
 {'ent': ['man', 'car', 'part', 'commercial'], 'rel': ['in', 'as', 'of']},
 {'ent': ['man'], 'rel': []},
 {'ent': ['man', 'car', 'road'], 'rel': ['ride', 'in']},
 {'ent': ['man', 'feature', 'car'], 'rel': ['show', 'of']},
 {'ent': ['man', 'experience', 'audi'], 'rel': ['narrate', 'drive']},
 {'ent': ['person', 'car', 'curve', 'road'], 'rel': ['drive', 'around', 'in']},
 {'ent': ['person', 'car'], 'rel': ['about']},
 {'ent': ['guy', 'car', 'road'], 'rel': ['drive', 'down']},
 {'ent': ['man', 'car'], 'rel': ['about']},
 {'ent': ['man', 'car'], 'rel': ['drive']},
 {'ent': ['man', 'audi'], 'rel': ['drive']},
 {'ent': ['man'], 'rel': []},
 {'ent': ['guy', 'car', 'road']

In [68]:
'> chair'.encode() in vocab

False

In [69]:
[t.lemma_ for t in nlp('> chair')][0]

'>'